In [853]:
from sentence_transformers import SentenceTransformer, util
# import mammoth # THIS IS FOR CONVERTING THE DOCX TO MD
from os import path, listdir
import re

In [854]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

In [855]:
%run load_docx.ipynb

In [856]:
file = '../Experimenters_SWY.docx'

In [857]:
file_name, file_extension = path.splitext(file)

In [858]:
if file_extension == '.docx':
    # doc = processDoc(file)
    md = processMD(file)
    # with open(file, "rb") as docx_file:
    #     md = mammoth.convert_to_markdown(docx_file)
        # len(doc[2:3][0])
print(md.messages)

[]


In [859]:
doc = md.value.split('\n\n')

In [860]:
def embed (string):
    return model.encode(string)

In [861]:
doc_embeddings = [(s,embed(s)) for s in doc]

In [862]:
def getType (md, w):
    principle = 'Purpose/ reason: Why is this important?'
    tactic = r'Tactic description: When do you apply this tactic\? How does it work\?'
    skill = r'Skills description: What does the ability help you do, achieve, or produce\?'
    tool = r'(Type of tool: If it is a tool, what type of tool is it\?|Short description: Describe in one sentence what the tool\/ method is, and what it does\.|Description of the tool: what is it\?)'

    res_principles = len(re.findall(principle, md))
    res_tactics = len(re.findall(tactic, md))
    res_skills = len(re.findall(skill, md))
    res_tools = len(re.findall(tool, md))

    # if res_principles == 0 and res_tactics == 0 and res_skills == 0 and res_tools == 0:
    print(w)
    print('pinciples', res_principles)
    print('tactics', res_tactics)
    print('skills', res_skills)
    print('tools', res_tools)
    print('\n')

    l = [('principles', res_principles), ('tactics', res_tactics), ('skills', res_skills), ('tools', res_tools)]
    return [t for (t,c) in l if c == max(res_principles, res_tactics, res_skills, res_tools)]

In [863]:
def filterMD (md):
    # REMOVE ALL THE CHECKLISTS
    md = re.sub(r'\#\#\#\# [\w\s\:\?\/\(\)\\;&]+\n', '', md)
    md = re.sub(r'\#+', '', md)
    md = re.sub(r'\n-[\s_]+(Sense|Explore|Develop|Test|Diffuse|Catalyze)', '', md)
    md = re.sub(r'\n-[\s_]+(Exploration|Solutions mapping|Experimentation)', '', md)
    md = re.sub(r'\n-[\s_]+(The advocate|The analyst|The communicator|The convener|The hustler|The project manager|The qualitative researcher|The visionary)', '', md)
    md = re.sub(r'\n-[\s_]+(Framework, Model|Guide, Manual|Platform, Portal|Playbook, Toolkit|Worksheet, Canvas|Process|Pilots|Proof of concept|Prototyping)', '', md)
    md = re.sub(r'\n-[\s_]+(Yes|No)', '', md)
    md = re.sub(r'[_\\]+', ' ', md)
    md = re.sub(r'\s+', ' ', md)
    return md.strip()

In [864]:
fromDB = True

In [865]:
if fromDB == True:
    %run get_keywords.ipynb
else:
    source = '/Users/myjyby/Documents/Projects/Codification fest/Transcripts/Elements/Docs/'
    seed_keywords = [(f, processMD(path.join(source, f)).value) for f in listdir(source) if f != '.DS_Store']

/var/folders/19/g73r505957gcy4y0smw3kd6r0000gn/T/ipykernel_6226/2256331286.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query,con=conn)


In [866]:
if fromDB == False:
    seed_keywords = [(w, filterMD(b), getType(b, w)[0]) for (w, b) in seed_keywords]

In [867]:
# print([(w,t) for (w,b,t) in seed_keywords])

In [868]:
for w,b,t in seed_keywords:
    if len(b) < 50:
        print(w, len(b))

seed_keywords = [(w,b,t) for (w,b,t) in seed_keywords if len(b) > 50]

In [869]:
seed_keywords[11]

('KoboToolbox for Mapping Solutions Nationwide',
 'KoboToolbox for Mapping Solutions NationwideKoboToolbox for Mapping Solutions Nationwide\n\nImage: What does it look like?\n\nReference: Where can we find more information or download materials?\nhttps://eu.kobotoolbox.org/accounts/login/\n\nDescription: What is it? (max 1,500 characters)\nKoboToolbox is an open-source platform for the collection, management, and visualization of data.\n\nPurpose: Why do we use this method or tool for R&amp;D? (max 1,500 characters)\nIt offers intuitive usability, tailored for efficient data collection without extensive technical training. Designed for challenging environments like humanitarian crises, it supports offline use on affordable mobile devices while providing advanced data management features. KoboToolbox allows easy migration between platforms, and supports multilingual survey forms, facilitating collaboration across global teams.\n\nSummary: Describe in one sentence, what it is and what it

In [870]:
embedTitle = False

if embedTitle:
    keyword_embeddings = [(w,t,embed(w)) for (w,b,t) in seed_keywords]
else:
    keyword_embeddings = [(w,t,embed(b)) for (w,b,t) in seed_keywords]

In [871]:
def compare (e1, e2):
    return util.pytorch_cos_sim(e1, e2).tolist()[0][0]

In [872]:
topTags = []
useTypes = False

output_string = ''

for snippet in doc_embeddings:
    if len(snippet[0]) > 100 and not snippet[0].startswith('#') :
        distances = [(w,t,compare(e, snippet[1])) for (w,t,e) in keyword_embeddings]
        distances.sort(key = lambda tup: tup[2], reverse = True)
        types = list(set([t for (w,t,e) in keyword_embeddings]))
        
        if useTypes:
            for t in types:
                top_keyword = [w for (w, dt, e) in distances if dt == t][0]
                keyword, extension = path.splitext(top_keyword)
                output_string = output_string + "[[{}:{}]]".format(t.replace('capture card', '').strip().lower(), keyword)
                output_string = output_string + '\n'
        else:
            # print([(w, e) for (w, dt, e) in distances][0:3])
            top_keyword = [(w, dt) for (w, dt, e) in distances][:4]
            # output_string = output_string + "[[{}]]".format(top_keyword)
            for w, dt in top_keyword:
                keyword, extension = path.splitext(w)
                output_string = output_string + "[[{}:{}]]".format(dt, keyword)
                output_string = output_string + '\n'
            # output_string = output_string + '\n\n'

        output_string = output_string + snippet[0]
        output_string = output_string + '\n\n'
        
        topTags += [w for (w,t,e) in distances[:5]]
        output_string = output_string + '\n'
        # print(snippet[0], distances[:5])
    else:
        output_string = output_string + snippet[0]
        output_string = output_string + '\n\n'
print(output_string)

# Experimenter\-Eswatini\-Zandile Mthembu

###   
\[\[0:00\]\] Tell me your name and your title and say I give consent to this recording\.

[[Tools capture card:Signals scanning
]]
[[Tools capture card:Randomized Controlled Trials]]
[[Tools capture card:Key Informant Interviews]]
[[Tools capture card:Horizon scanning]]
I'm Zandile in Tembu\. I'm head of experimentation at UNDP from the Kingdom of a Swat Dini\. I give consent to this recording\.


###   
\[\[0:16\]\] Over to you and one other question\.

###   
\[\[0:49\]\] So can you tell me about an Rnd activity  you would like to share?

[[Tools capture card:Innovation Challenge]]
[[Skills capture card:Sharing Knowledge, training, and unusual partnership]]
[[Skills capture card:Collective intelligence, video, visibility was key success, "press advisor"]]
[[Tools capture card:Innovation Challenge merged with blended financing ]]
So I'd like to share an A most recent R&D activity that we're working on\. Currently in my country, there's

In [873]:
def findDuplicates (l):
    seen = set()
    duplicates = []

    for x in l:
        if x in seen and x not in duplicates:
            duplicates.append(x)
        else:
            seen.add(x)
    return duplicates

In [874]:
findDuplicates(topTags)

['Innovation Challenge',
 'Innovation Challenge merged with blended financing\xa0',
 'Never deplete social capital, always increase social capital.',
 'Scaling partnerships by demonstrating value incrementally',
 'Sharing Knowledge, training, and unusual partnership',
 'Even when the Lab is not there (or UNDP) things should continue - we have t…',
 'Collaborative design of actions. Collective intelligence for experimenting and scaling with others…',
 'Citizen Science and gamification using digital platform to crowdsource solutions',
 'Human centering and adaptability: Less of me, more of them',
 'Collaboration',
 'Recognize and promote agency: Enable agency, the greatest choice for the greatest number.',
 'Collective intelligence, video, visibility was key success, "press advisor"']

In [875]:
output_name = "{}.md".format('_'.join([t for t in file_name.split('/') if t != '..' and t != '.']))
output_name

'Experimenters_SWY.md'

In [876]:
output_file = open("../stories/pages/{}".format(output_name), 'w')
output_file.write(output_string)
output_file.close()